# GREAT CARIA v3.0 – Improved Robust Implementation

Esta versión incorpora todas las mejoras sugeridas para fortalecer la validación y aumentar la señal predictiva del modelo.  Incluye:

- Uso de un objetivo de cambio de dirección del Factor de Crisis (CF) a un horizonte configurable (por defecto 5 días).
- Generación de nuevas variables que reflejan **momentum**, **volatilidad realizada** y **factores macro** (VIX, DXY, Oil, Gold).
- Expansión a múltiples países (países seleccionados por disponibilidad de datos en el parquet).
- División purgada y con embargo para evitar fugas de información.
- Modelo base de **Regresión Logística** con ponderación de clases y evaluación mediante **Accuracy** y **AUC**.
- Prueba de barajado (shuffle) para descartar señales espurias.
- Validación out‑of‑sample (rolling OOS) con medidas de desempeño por fold.
- Ejemplo de modelo **Random Forest** para comparar con la regresión logística.

***Nota***: para ejecutar este cuaderno necesitas tener acceso a los datos originales (archivos parquet) y modificar la ruta `MARKET_PATH` según tu entorno.


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
# === 1. CARGA DE DATOS ===
# Ajusta la ruta al archivo parquet según tu entorno
MARKET_PATH = '/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet'

# Lectura del dataset de mercado diario
df_daily = pd.read_parquet(MARKET_PATH)

# Lista de países a utilizar (puedes ampliarla si hay datos disponibles)
COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'BRA', 'MEX', 'KOR', 'AUS']

print(f'Datos diarios cargados: {df_daily.shape}')

In [ ]:
# === 2. CÁLCULO DEL FACTOR DE CRISIS (CF) ===
# Utilizamos retornos diarios de los índices de cada país
index_cols = [f'{c}_index' for c in COUNTRIES if f'{c}_index' in df_daily.columns]
returns = df_daily[index_cols].pct_change().dropna()
returns.columns = [c.replace('_index','') for c in returns.columns]

def compute_cf(returns, window=20):
    cf_series = []
    for i in range(window, len(returns)):
        w = returns.iloc[i-window:i]
        corr = w.corr().values
        # correlación media entre todos los pares
        avg_corr = (corr.sum() - len(corr)) / (len(corr) * (len(corr) - 1))
        avg_vol = w.std().mean()
        cf_series.append((avg_corr + avg_vol) / 2)
    cf_index = returns.index[window:]
    return pd.Series(cf_series, index=cf_index)

CF = compute_cf(returns)
print(f'CF calculado para {len(CF)} observaciones')

In [ ]:
# === 3. OBJETIVO: Cambio de dirección del CF ===
HORIZON = 5
cf_future = CF.shift(-HORIZON)
cf_change = (cf_future > CF).astype(int)  # 1 = CF aumentará
cf_change = cf_change.dropna()
print(f'Observaciones de CF para el objetivo: {len(cf_change)}')

In [ ]:
# === 4. INGENIERÍA DE CARACTERÍSTICAS ===
# Inicializamos dataframe de características
features = pd.DataFrame(index=cf_change.index)

# Variables relacionadas con CF
features['cf_now'] = CF.loc[cf_change.index]
features['cf_ma5'] = CF.rolling(5).mean().loc[cf_change.index]
features['cf_ma20'] = CF.rolling(20).mean().loc[cf_change.index]

# Factores macro globales (si están disponibles en df_daily)
for col in ['VIX', 'DXY', 'Oil', 'Gold']:
    if col in df_daily.columns:
        features[col.lower()] = df_daily[col].loc[cf_change.index]

# Variables de momentum y volatilidad por país
for c in COUNTRIES:
    if c in returns.columns:
        # Media móvil de 5 días
        features[f'ret_{c.lower()}_ma5'] = returns[c].rolling(5).mean().loc[cf_change.index]
        # Volatilidad realizada en 20 días
        features[f'vol_{c.lower()}_std20'] = returns[c].rolling(20).std().loc[cf_change.index]

# Eliminamos filas con valores faltantes
features = features.dropna()
# Reajustamos el objetivo a las fechas disponibles
target = cf_change.loc[features.index]

print(f'Número final de muestras: {len(features)}')

In [ ]:
# === 5. DIVISIÓN PURGADA Y EMBARGO ===
PURGE = 20
EMBARGO = 10
train_end = int(len(features) * 0.7)
test_start = train_end + PURGE + EMBARGO

X_train = features.iloc[:train_end].values
y_train = target.iloc[:train_end].values
X_test = features.iloc[test_start:].values
y_test = target.iloc[test_start:].values

# Normalización basada en el conjunto de entrenamiento
mu, sigma = X_train.mean(axis=0), X_train.std(axis=0) + 1e-8
X_train_norm = (X_train - mu) / sigma
X_test_norm = (X_test - mu) / sigma

print(f'Tamaño entrenamiento: {len(X_train_norm)}, test: {len(X_test_norm)} (purge={PURGE}, embargo={EMBARGO})')

In [ ]:
# === 6. REGRESIÓN LOGÍSTICA (con ponderación de clases) ===
# Se usa class_weight='balanced' para manejar posibles desequilibrios de clases
lr = LogisticRegression(max_iter=1000, C=0.1, class_weight='balanced')
lr.fit(X_train_norm, y_train)

# Predicciones de clase y probabilidades
y_pred_lr = lr.predict(X_test_norm)
y_pred_proba_lr = lr.predict_proba(X_test_norm)[:, 1]

acc_lr = accuracy_score(y_test, y_pred_lr)
auc_lr = roc_auc_score(y_test, y_pred_proba_lr)

print(f'Regresión logística – Accuracy (purged): {acc_lr:.3f}, AUC: {auc_lr:.3f}')

In [ ]:
# === 7. PRUEBA DE BARAJADO (shuffle) ===
y_train_shuffled = np.random.permutation(y_train)
lr_shuffled = LogisticRegression(max_iter=1000, C=0.1, class_weight='balanced')
lr_shuffled.fit(X_train_norm, y_train_shuffled)

y_pred_shuffle = lr_shuffled.predict(X_test_norm)
y_pred_proba_shuffle = lr_shuffled.predict_proba(X_test_norm)[:, 1]

acc_shuffle = accuracy_score(y_test, y_pred_shuffle)
auc_shuffle = roc_auc_score(y_test, y_pred_proba_shuffle)

print(f'Barajado – Accuracy: {acc_shuffle:.3f}, AUC: {auc_shuffle:.3f}')
print(f'Lift sobre random (accuracy): {(acc_lr - acc_shuffle):.3f}')

In [ ]:
# === 8. VALIDACIÓN ROLLING OOS ===
n_folds = 5
fold_size = len(features) // (n_folds + 1)
rolling_accs = []
rolling_aucs = []
for i in range(n_folds):
    train_end_i = (i + 1) * fold_size
    test_start_i = train_end_i + PURGE + EMBARGO
    test_end_i = test_start_i + fold_size
    if test_end_i > len(features):
        break
    X_tr = features.iloc[:train_end_i].values
    y_tr = target.iloc[:train_end_i].values
    X_te = features.iloc[test_start_i:test_end_i].values
    y_te = target.iloc[test_start_i:test_end_i].values
    mu_i, sigma_i = X_tr.mean(axis=0), X_tr.std(axis=0) + 1e-8
    X_trn = (X_tr - mu_i) / sigma_i
    X_ten = (X_te - mu_i) / sigma_i
    lr_fold = LogisticRegression(max_iter=1000, C=0.1, class_weight='balanced')
    lr_fold.fit(X_trn, y_tr)
    y_pred_fold = lr_fold.predict(X_ten)
    y_proba_fold = lr_fold.predict_proba(X_ten)[:, 1]
    acc_fold = accuracy_score(y_te, y_pred_fold)
    auc_fold = roc_auc_score(y_te, y_proba_fold)
    rolling_accs.append(acc_fold)
    rolling_aucs.append(auc_fold)
    print(f'  Fold {i+1}: Acc {acc_fold:.3f}, AUC {auc_fold:.3f}')

print(f'
Rolling OOS – media Accuracy: {np.mean(rolling_accs):.3f}, std: {np.std(rolling_accs):.3f}')
print(f'Rolling OOS – media AUC: {np.mean(rolling_aucs):.3f}, std: {np.std(rolling_aucs):.3f}')

In [ ]:
# === 9. RANDOM FOREST COMO MODELO ALTERNATIVO ===
# Entrenamos un RandomForestClassifier simple para comparar
rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42, class_weight='balanced')
rf.fit(X_train_norm, y_train)

y_pred_rf = rf.predict(X_test_norm)
y_proba_rf = rf.predict_proba(X_test_norm)[:, 1]

acc_rf = accuracy_score(y_test, y_pred_rf)
auc_rf = roc_auc_score(y_test, y_proba_rf)

print(f'Random Forest – Accuracy: {acc_rf:.3f}, AUC: {auc_rf:.3f}')

In [ ]:
# === 10. RESUMEN ===
print('
' + '='*60)
print('Resultados de VALIDACIÓN – GREAT CARIA v3.0')
print('='*60)
print(f'Regresión logística – Accuracy: {acc_lr:.3f}, AUC: {auc_lr:.3f}')
print(f'Modelo barajado – Accuracy: {acc_shuffle:.3f}, AUC: {auc_shuffle:.3f}')
print(f'Lift sobre random (accuracy): {(acc_lr - acc_shuffle):.3f}')
print(f'Rolling OOS – media Acc: {np.mean(rolling_accs):.3f}, media AUC: {np.mean(rolling_aucs):.3f}')
print(f'Random Forest – Accuracy: {acc_rf:.3f}, AUC: {auc_rf:.3f}')

# Veredicto simple basándonos en accuracy y AUC
if acc_lr > 0.55 and auc_lr > 0.55 and (acc_lr - acc_shuffle) > 0.03:
    print('✓ PASADO – Se detecta señal estadísticamente significativa')
else:
    print('✗ FALLA – La señal es marginal o no supera el azar con margen suficiente')
print('='*60)